In [1]:
from tools import *
import boto3
brt = boto3.client('bedrock-runtime')

## ユーザーのクエリを深ぼる

In [2]:
USER_PROMPT = '主要なクラウドサービス(AWS, Azure, GCP)の生成 AI 領域における比較'
MODEL_ID_DICT = {
    'claude-3.5-sonnet': 'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
    'nova-pro': 'us.amazon.nova-pro-v1:0',
    'mistral-large': 'mistral.mistral-large-2407-v1:0',
    'llama-3.3' : 'us.meta.llama3-3-70b-instruct-v1:0'
}

# 使用するモデルのキーを設定
MODEL_A = 'claude-3.5-sonnet'  # 1つ目のAI
MODEL_I = 'llama-3.3'      # 2つ目のAI

MAX_LOOP = 5

QUERY_QUALIFICATION_SYSTEM_PROMPTS = [{
    'text':f'''あなたは優秀なリサーチャーです。
会話相手はあなたと同じ {USER_PROMPT} という調査依頼を受けとった同僚の AI さんです。
調査内容は雑なので、調査の粒度や観点などは仮説を持って調査をした後調査結果を作成し、調査結果のフィードバックをもらうことでしか改善できません。
AI さんはあなたの思考の枠を外して広い視野を提供してくれます。
あなたも調査内容を自由に広げて網羅性を高め、その後どんなことを調べるのかを深めていってください。
特に反対意見は大事です。お互いの網羅性や深さの不足を指摘しながらAI さんとの会話を重ね、リサーチする内容を決めていってください。
会話はお互い {MAX_LOOP} 回までしかできないので、それまでに議論をまとめてください。
ただし会話は <rules> で与える内容をまとめてください。
<rules>
* 具体的にどんなことをするのか actionable にまとめる必要があります。
* 予算や人員については触れてはいけません。調査する内容にだけフォーカスしてください。
* 調査対象に関わるだろう人の観点を複数入れてください。例えば料理であれば、調理器具をつくる人、食材を運ぶ人、料理を作る人、料理を運ぶ人、食べる人、口コミを書く人などです。与えられたお題に反しない限り様々な人に思いを巡らせてください。
* 会話を始める前に、自分がどのように考えたのか、を述べてから結論を述べてください。
* さまざまな観点から内容をブラッシュアップしてください。
</rules>
また、発言する際は最初に必ず x 回目の発言です、と言ってください。発言回数は自分の発言回数であり、相手の発言はカウントしてはいけません。
'''}]

conversation = {
    'A':[
        {
            "role": "user",
            "content": [{"text": f'「{USER_PROMPT}」って大変なお題をもらっちゃいましたね。どうしましょうか。'}]
        }
    ],
    'I':[]
}

# 5往復のループ
for i in range(MAX_LOOP):
    # 1つ目のAIの応答
    response = brt.converse(
        modelId=MODEL_ID_DICT[MODEL_A],
        messages=conversation['A'],
        system=QUERY_QUALIFICATION_SYSTEM_PROMPTS,
        inferenceConfig={'temperature': 1},
    )
    first_response = response['output']['message']['content'][0]['text']
    print(f"\n=== {MODEL_A}:{i+1} 回目の発言 ===\n")
    print(first_response)
    
    
    conversation['A'].append(
        {
            "role": "assistant",
            "content": [{"text": first_response}]
        }
    )
    conversation['I'].append(
        {
            "role": "user",
            "content": [{"text": first_response}]
        }
    )

    # 2つ目のAIの応答
    response = brt.converse(
        modelId=MODEL_ID_DICT[MODEL_I],
        messages=conversation['I'],
        system=QUERY_QUALIFICATION_SYSTEM_PROMPTS,
        inferenceConfig={'temperature': 1},
    )
    second_response = response['output']['message']['content'][0]['text']
    print(f"\n=== {MODEL_I}:{i+1} 回目の発言 ===\n")
    print(second_response)
    
    # 次の往復のために会話履歴を更新
    conversation['I'].append(
        {
            "role": "assistant",
            "content": [{"text": second_response}]
        }
    )
    conversation['A'].append(
        {
            "role": "user",
            "content": [{"text": second_response}]
        }
    )


=== claude-3.5-sonnet:1 回目の発言 ===

1回目の発言です。

まず、このテーマについて以下のように考えました：

1. 生成AIの領域は急速に発展している分野なので、単純な機能比較だけでは不十分
2. 各クラウドプロバイダーの強みや特徴が異なるため、多角的な視点が必要
3. 実際の利用者の視点を重視する必要がある

そこで、以下の観点から調査を進めることを提案します：

【関係者の視点】
- システム開発者（APIの使いやすさ、開発ツールの充実度）
- MLエンジニア（モデルのカスタマイズ性、学習環境）
- ビジネス意思決定者（コスト効率、ビジネス価値）
- エンドユーザー（使い勝手、応答速度）
- セキュリティ担当者（データ保護、コンプライアンス）

【調査項目】
1. 技術面
- 提供されている生成AIモデルの種類と特徴
- APIの充実度と使いやすさ
- カスタマイズ可能性
- 統合されているサービスの範囲

2. 運用面
- モニタリングと管理機能
- セキュリティ機能
- スケーラビリティ
- サポート体制

3. エコシステム
- サードパーティ連携
- コミュニティの活性度
- 事例やドキュメントの充実度

この調査方針について、見落としている視点や、より重要な観点があればご指摘いただけますでしょうか？

=== llama-3.3:1 回目の発言 ===

2回目の発言です。

あなたの提案は包括的かつ実践的な観点を包含しています。以下にいくつか補足的な考察と提案を述べます：

1. **倫理と説明責任**: 生成AIのエチケットや倫理的な使用、モデルの説明責任（説明可能性）についても調査する必要があるかもしれません。これは、AIの使いやすさだけでなく、利用者や社会への影響を考慮する必要があるからです。

2. **ユーザーインターフェイスのアクセシビリティ**: エンドユーザーの観点から、アクセシビリティやユーザー体験を含めて考えることが重要です。特に、視覚障害を持つユーザーなど、さまざまな障害を持つユーザーを含めたアクセシビリティの考慮が必要です。

3. **データ品質とプライバシー**: データ保護とプライバシーに関する観点をさらに深掘りする価値があります。特に、生成AIで利用されるデータの管理、データ漏洩対策

## 調査すべき内容、アプローチをまとめる

In [3]:
SURVEY_STRATEGY_PROMPTS = [{
    'text':f'''あなたは優秀なリサーチャーです。
あなたは「{USER_PROMPT}」 という調査依頼を受けとっています。
調査内容は雑なので、調査の粒度や観点などは仮説を持って調査をした後調査結果を作成し、調査結果のフィードバックをもらうことでしか改善できません。
どのように調査を進めるかの方針をまとめた会話を渡します。
調査の方針をまとめてください。
ただし出力する内容は調査の方針だけで、会話を続ける必要はありません。
'''}]

In [4]:
response = brt.converse(
    modelId=MODEL_ID_DICT[MODEL_A],
    messages=conversation['A'],
    system=SURVEY_STRATEGY_PROMPTS,
    inferenceConfig={'temperature': 0},
)
report_framework_response = response['output']['message']['content'][0]['text']
print(report_framework_response)

調査の方針を以下のようにまとめます：

【調査フレームワーク】

1. 技術評価（Technical Assessment）
- 基本機能比較（モデル種類、性能、APIデザイン）
- 拡張性評価（カスタマイズ性、スケーラビリティ）
- 統合機能（他サービスとの連携、既存システムとの統合）
- アクセシビリティ（多言語対応、障害者対応）

2. 運用評価（Operational Assessment）
- デプロイメントオプション
- モニタリング・管理機能
- セキュリティ機能
- コスト最適化機能
- サステナビリティ指標

3. ガバナンス（Governance）
- コンプライアンス対応
- プライバシー保護
- 監査機能
- 認証取得状況
- 倫理的ガイドライン

4. エコシステム（Ecosystem）
- パートナーシップ
- コミュニティ活動
- 教育リソース
- サポート体制
- フィードバックループ

5. 戦略的評価（Strategic Assessment）
- 開発ロードマップ
- 更新頻度と影響
- ベンダーロックインリスク
- オープン標準への対応
- 市場での位置づけ

【評価手法】

1. ベンチマーク評価
- 性能測定
- 機能比較
- コスト効率性

2. ユーザー調査
- 開発者フィードバック
- エンドユーザー評価
- セキュリティ担当者の評価

3. 事例分析
- 成功事例
- 失敗事例
- 業界別の適用性

4. エキスパートレビュー
- 技術専門家の評価
- セキュリティ専門家の評価
- 業界アナリストの見解


In [5]:
SURVEY_STRATEGY_PROMPTS = [{
    'text':f'''あなたは優秀なリサーチャーです。
あなたは「{USER_PROMPT}」 という調査依頼を受けとっています。
ユーザーはレポートのフレームワーク与えます。
ただしあなたは Web 検索をするか、検索結果の URL にアクセスをして情報を取得し、その結果を元に自分自身で考察する以外のことはできません。
あなたは、どうやって調査を進めるか考えたかと保持しているツールを使用して、レポートを完成させてください。
'''}]

In [6]:
tools = []
tools.append(
    {
        'toolSpec': {
            'name': 'search',
            'description': '''検索する文章、キーワードを受け取って Web 検索する''',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'string',
                            'description': '検索する文章またはキーワード。半角スペースで区切ることで複数のキーワードを受け付ける。',
                        }
                    },
                    'required': ['query'],
                }
            },
        }
    }
)
tools.append(
    {
        'toolSpec': {
            'name': 'get_html',
            'description': '''URL にアクセスして html のコンテンツを取得''',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {
                        'url': {
                            'type': 'string',
                            'description': '情報を取得したい URL',
                        }
                    },
                    'required': ['url'],
                }
            },
        }
    }
)
tools.append(
    {
        'toolSpec': {
            'name': 'is_finished',
            'description': '''ツールの利用が完了し、次のステップに進む関数''',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {},
                    'required': [],
                }
            },
        }
    }
)
tool_config = {'tools': tools}

In [11]:
conversation['F'] = []
conversation['F'].append(
    {
        "role": "user",
        "content": [{"text": report_framework_response}]
    }
)
print(conversation['F'])

[{'role': 'user', 'content': [{'text': '調査の方針を以下のようにまとめます：\n\n【調査フレームワーク】\n\n1. 技術評価（Technical Assessment）\n- 基本機能比較（モデル種類、性能、APIデザイン）\n- 拡張性評価（カスタマイズ性、スケーラビリティ）\n- 統合機能（他サービスとの連携、既存システムとの統合）\n- アクセシビリティ（多言語対応、障害者対応）\n\n2. 運用評価（Operational Assessment）\n- デプロイメントオプション\n- モニタリング・管理機能\n- セキュリティ機能\n- コスト最適化機能\n- サステナビリティ指標\n\n3. ガバナンス（Governance）\n- コンプライアンス対応\n- プライバシー保護\n- 監査機能\n- 認証取得状況\n- 倫理的ガイドライン\n\n4. エコシステム（Ecosystem）\n- パートナーシップ\n- コミュニティ活動\n- 教育リソース\n- サポート体制\n- フィードバックループ\n\n5. 戦略的評価（Strategic Assessment）\n- 開発ロードマップ\n- 更新頻度と影響\n- ベンダーロックインリスク\n- オープン標準への対応\n- 市場での位置づけ\n\n【評価手法】\n\n1. ベンチマーク評価\n- 性能測定\n- 機能比較\n- コスト効率性\n\n2. ユーザー調査\n- 開発者フィードバック\n- エンドユーザー評価\n- セキュリティ担当者の評価\n\n3. 事例分析\n- 成功事例\n- 失敗事例\n- 業界別の適用性\n\n4. エキスパートレビュー\n- 技術専門家の評価\n- セキュリティ専門家の評価\n- 業界アナリストの見解'}]}]


In [ ]:
for i in range(10):
    print(i)
    response = brt.converse(
        modelId=MODEL_ID_DICT[MODEL_A],
        messages=conversation['F'],
        system=SURVEY_STRATEGY_PROMPTS,
        inferenceConfig={'temperature': 0},
        toolConfig=tool_config
    )
    conversation['F'].append({
        'role': 'assistant', 
        'content': response['output']['message']['content']
    })
    tool_use = response.get('output', {}).get('message', {}).get('content', [])[1].get('toolUse')
    if tool_use is not None:
        function_name = tool_use['name']
        if function_name == 'is_finished':
            break
        function_args = tool_use['input']
        
        result = globals()[function_name](**function_args)
        conversation['F'].append({
            'role': 'user', 
            'content': [
                {
                    'toolResult':{
                        'toolUseId':tool_use['toolUseId'],
                        'content':[
                            {
                                'type':'text',
                                'text': result
                            }
                        ]
                    }
                }
            ]
        })
    else:
        break


0
1


ParamValidationError: Parameter validation failed:
Invalid number of parameters set for tagged union structure messages[2].content[0].toolResult.content[0]. Can only set one of the following keys: json, text, image, document, video.
Unknown parameter in messages[2].content[0].toolResult.content[0]: "type", must be one of: json, text, image, document, video
Invalid type for parameter messages[2].content[0].toolResult.content[0].text, value: {'content': '[{"title": "Generative AI Cloud Platforms: AWS, Azure, or Google? - The New Stack", "url": "https://thenewstack.io/generative-ai-cloud-services-aws-azure-or-google-cloud/", "description": "<strong>Generative</strong> <strong>AI</strong> needs massive computing power and large datasets, which makes the public cloud an ideal platform choice. But who will dominate? Amazon Web Services, <strong>Azure</strong> or Google Cloud?"}, {"title": "Generative AI Offerings and LLMs: A Comparison of ...", "url": "https://www.predera.com/blog/generative

In [18]:
result

'[{"title": "Generative AI Cloud Platforms: AWS, Azure, or Google? - The New Stack", "url": "https://thenewstack.io/generative-ai-cloud-services-aws-azure-or-google-cloud/", "description": "<strong>Generative</strong> <strong>AI</strong> needs massive computing power and large datasets, which makes the public cloud an ideal platform choice. But who will dominate? Amazon Web Services, <strong>Azure</strong> or Google Cloud?"}, {"title": "Generative AI Offerings and LLMs: A Comparison of ...", "url": "https://www.predera.com/blog/generative-ai-offerings-and-llms-a-comparison-of-google-vs-azure-vs-amazon", "description": "REQUEST DEMO · blog posts and used cases · What we think is in our BLOG · New Partnership Announcement - Aarna Networks · Marketing Team · March 9, 2023 · read more · Hive to HBase Migration · Siva Kumar Reddy · March 27, 2023"}, {"title": "Artificial Intelligence and Machine Learning: AWS vs Azure vs GCP", "url": "https://www.pluralsight.com/resources/blog/cloud/aws-vs-

In [17]:
conversation['F'][2]['content'][0]['toolResult']['content'][0]['text']

{'content': '[{"title": "Generative AI Cloud Platforms: AWS, Azure, or Google? - The New Stack", "url": "https://thenewstack.io/generative-ai-cloud-services-aws-azure-or-google-cloud/", "description": "<strong>Generative</strong> <strong>AI</strong> needs massive computing power and large datasets, which makes the public cloud an ideal platform choice. But who will dominate? Amazon Web Services, <strong>Azure</strong> or Google Cloud?"}, {"title": "Generative AI Offerings and LLMs: A Comparison of ...", "url": "https://www.predera.com/blog/generative-ai-offerings-and-llms-a-comparison-of-google-vs-azure-vs-amazon", "description": "REQUEST DEMO · blog posts and used cases · What we think is in our BLOG · New Partnership Announcement - Aarna Networks · Marketing Team · March 9, 2023 · read more · Hive to HBase Migration · Siva Kumar Reddy · March 27, 2023"}, {"title": "Artificial Intelligence and Machine Learning: AWS vs Azure vs GCP", "url": "https://www.pluralsight.com/resources/blog/c